# ITU 5G Energy Consumption Challenge - GitData

<font color="red">Please make a copy of this Jupyter Notebook to your own Google Drive or local machine before changing anything.</font>

This Jupyter Notebook will also be updated regularly on GitHub: https://github.com/GitData-GA/itu5g/tree/main/code/ITU_5G_Energy_Consumption_Challenge.ipynb

In [1]:
# Import all libraries here
import random
random.seed(88)
import pandas as pd
import numpy as np
np.random.seed(88)
import tensorflow as tf
tf.random.set_seed(88)
from tensorflow import keras
keras.utils.set_random_seed(88)
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1, l2
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
# Output format
# Description: The file contains days, hours, and base stations for which we
# need energy consumption estimation and will be used for scoring predictions.
power_pred = pd.read_csv('https://data.gd.edu.kg/bwz7t527/power_consumption_prediction.csv')
# Dataset 1
# Description: Energy consumption data (ECdata.csv): include hour-level energy
# consumption specifications (e.g., total energy consumption of the base stations).
ECdata = pd.read_csv('https://data.gd.edu.kg/bwz7t527/ECdata.csv')
# Dataset 2
# Description: Cell-level data (CLdata.csv): include hour-level counters,
# including service compliance counters (e.g., load) and energy-saving methods
# counters (e.g., duration of energy saving mode activation).
CLdata = pd.read_csv('https://data.gd.edu.kg/bwz7t527/CLdata.csv')
# Dataset 3
# Description: Base Station basic information (BSinfo.csv): include
# configuration parameters and hardware attributes
BSinfo = pd.read_csv('https://data.gd.edu.kg/bwz7t527/BSinfo.csv')

In [3]:
data = ECdata.merge(CLdata, on=["Time", "BS"], how="inner")
data = data.merge(BSinfo, on="BS", how="inner")
data = data.sort_values(by='Time', ascending=True)
# Convert date strings to datetime objects
data['Time'] = pd.to_datetime(data['Time'])

# Sort the dataset by date
data.sort_values(by='Time', inplace=True)

# Split Year, Month, Day, and Hour
data['Year'] = data['Time'].dt.year
data['Month'] = data['Time'].dt.month
data['Day'] = data['Time'].dt.day
data['Hour'] = data['Time'].dt.hour

# Extract target variable 'Energy'
y = data['Energy']

# Drop unnecessary columns (e.g., 'Time', 'Energy')
data.drop(columns=['Time', 'Energy'], inplace=True)

# One-hot encode categorical features
categorical_cols = ['BS', 'CellName_x', 'CellName_y', 'RUType', 'Mode', ]
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

# Standardize numerical features
scaler = StandardScaler()
data_encoded[data_encoded.columns] = scaler.fit_transform(data_encoded[data_encoded.columns])

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data_encoded, y, test_size=0.2, shuffle=False)

# Convert data to numpy arrays
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
# Time series transformation
X_train_ts = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_ts = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [4]:
# LSTM Model - Time series
lstm_model = keras.Sequential([
    layers.LSTM(units=128, recurrent_dropout=0.125),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
lstm_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = lstm_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = lstm_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

lstm_model.summary()

Epoch 1/50
2514/2514 [==============================] - 42s 14ms/step - loss: 26.4283 - mean_absolute_percentage_error: 26.4283 - val_loss: 8.8147 - val_mean_absolute_percentage_error: 8.8147
Epoch 2/50
2514/2514 [==============================] - 36s 14ms/step - loss: 14.1385 - mean_absolute_percentage_error: 14.1385 - val_loss: 7.5431 - val_mean_absolute_percentage_error: 7.5431
Epoch 3/50
2514/2514 [==============================] - 34s 14ms/step - loss: 12.5811 - mean_absolute_percentage_error: 12.5811 - val_loss: 7.7807 - val_mean_absolute_percentage_error: 7.7807
Epoch 4/50
2514/2514 [==============================] - 34s 14ms/step - loss: 11.6646 - mean_absolute_percentage_error: 11.6646 - val_loss: 7.0439 - val_mean_absolute_percentage_error: 7.0439
Epoch 5/50
2514/2514 [==============================] - 33s 13ms/step - loss: 10.9172 - mean_absolute_percentage_error: 10.9172 - val_loss: 6.4138 - val_mean_absolute_percentage_error: 6.4138
Epoch 6/50
2514/2514 [==================

In [5]:
# GRU Model - Time series
gru_model = keras.Sequential([
    layers.GRU(units=128, recurrent_dropout=0.125),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
gru_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = gru_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = gru_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

gru_model.summary()

Epoch 1/50
2514/2514 [==============================] - 44s 15ms/step - loss: 25.4439 - mean_absolute_percentage_error: 25.4439 - val_loss: 8.8262 - val_mean_absolute_percentage_error: 8.8262
Epoch 2/50
2514/2514 [==============================] - 32s 13ms/step - loss: 14.2679 - mean_absolute_percentage_error: 14.2679 - val_loss: 8.4414 - val_mean_absolute_percentage_error: 8.4414
Epoch 3/50
2514/2514 [==============================] - 33s 13ms/step - loss: 12.5403 - mean_absolute_percentage_error: 12.5403 - val_loss: 8.2822 - val_mean_absolute_percentage_error: 8.2822
Epoch 4/50
2514/2514 [==============================] - 31s 12ms/step - loss: 11.5365 - mean_absolute_percentage_error: 11.5365 - val_loss: 7.1606 - val_mean_absolute_percentage_error: 7.1606
Epoch 5/50
2514/2514 [==============================] - 33s 13ms/step - loss: 10.5589 - mean_absolute_percentage_error: 10.5589 - val_loss: 6.6687 - val_mean_absolute_percentage_error: 6.6687
Epoch 6/50
2514/2514 [==================

In [6]:
# RNN Model - Time series
rnn_model = keras.Sequential([
    layers.SimpleRNN(units=128, recurrent_dropout=0.125),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
rnn_model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = rnn_model.fit(X_train_ts, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = rnn_model.predict(X_test_ts)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

rnn_model.summary()

Epoch 1/50
2514/2514 [==============================] - 30s 10ms/step - loss: 26.1427 - mean_absolute_percentage_error: 26.1427 - val_loss: 8.7998 - val_mean_absolute_percentage_error: 8.7998
Epoch 2/50
2514/2514 [==============================] - 27s 11ms/step - loss: 13.8888 - mean_absolute_percentage_error: 13.8888 - val_loss: 7.5423 - val_mean_absolute_percentage_error: 7.5423
Epoch 3/50
2514/2514 [==============================] - 26s 10ms/step - loss: 12.5567 - mean_absolute_percentage_error: 12.5567 - val_loss: 7.2809 - val_mean_absolute_percentage_error: 7.2809
Epoch 4/50
2514/2514 [==============================] - 24s 10ms/step - loss: 11.8082 - mean_absolute_percentage_error: 11.8082 - val_loss: 6.7454 - val_mean_absolute_percentage_error: 6.7454
Epoch 5/50
2514/2514 [==============================] - 24s 10ms/step - loss: 11.0102 - mean_absolute_percentage_error: 11.0102 - val_loss: 6.5652 - val_mean_absolute_percentage_error: 6.5652
Epoch 6/50
2514/2514 [==================

In [7]:
# Regression Model - Regression
model = keras.Sequential([
    layers.Dense(units=128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=32, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=16, activation='relu'),
    layers.Dense(units=1, activation='linear')
])

# Compile the model
model.compile(optimizer='Adam', loss='mean_absolute_percentage_error',
                   metrics=['mean_absolute_percentage_error'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks = [callback])

# Fit model on validation set and get error
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'MAPE on test set: {100 * mape:.5f}')

model.summary()

Epoch 1/50
2514/2514 [==============================] - 22s 7ms/step - loss: 26.6295 - mean_absolute_percentage_error: 26.6295 - val_loss: 8.6296 - val_mean_absolute_percentage_error: 8.6296
Epoch 2/50
2514/2514 [==============================] - 17s 7ms/step - loss: 14.3137 - mean_absolute_percentage_error: 14.3137 - val_loss: 7.7987 - val_mean_absolute_percentage_error: 7.7987
Epoch 3/50
2514/2514 [==============================] - 18s 7ms/step - loss: 12.8631 - mean_absolute_percentage_error: 12.8631 - val_loss: 7.5072 - val_mean_absolute_percentage_error: 7.5072
Epoch 4/50
2514/2514 [==============================] - 17s 7ms/step - loss: 11.9588 - mean_absolute_percentage_error: 11.9588 - val_loss: 6.9287 - val_mean_absolute_percentage_error: 6.9287
Epoch 5/50
2514/2514 [==============================] - 18s 7ms/step - loss: 11.1623 - mean_absolute_percentage_error: 11.1623 - val_loss: 6.7759 - val_mean_absolute_percentage_error: 6.7759
Epoch 6/50
2514/2514 [=======================